In [1]:
import cv2
import numpy as np
import os
import copy

picsPath = "./pics/"        #location of images to apply similary transform on (all files in this location will be processed)
resultPath = "./out/"       #final transformed image path
modelImage = "model.jpg"    #located in root folder, the ideal placement of the object you're trying to match up to

imgSize = 512               #final output image size (square)
workingSize = 1024          #working image size (what's shown on screen)

In [2]:
outputPts = np.float32([[0,0],[0,0]])

class transformImage:
    def __init__(self, file):
        self.model = True if file == modelImage else False
        self.filePath = picsPath + file if self.model == False else file
        self.savePath = resultPath + file
        self.clickIndex = 0
        self.inputPts = np.float32([[0,0],[0,0]])
        global outputPts

        #load the image
        self.img = cv2.imread(self.filePath, 1)

        #pad the image to make square
        self.img = self.padImg(self.img)     

        #resize image to working size
        self.img = cv2.resize(self.img, (workingSize,workingSize), interpolation = cv2.INTER_AREA)

        #display image in window and wait for clicks
        self.displayImg = copy.deepcopy(self.img)
        if self.model == True:
            cv2.putText(self.displayImg, 'Set Output Points', (20,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        cv2.imshow('image', self.displayImg)
        cv2.setMouseCallback('image', self.mouseEvent)
        cv2.waitKey(0)

    #Perform transformations - don't save images that do not have transformations
        if self.model == True:
            outputPts = self.inputPts
        elif np.array_equal(self.inputPts, np.float32([[0,0],[0,0]])) == False:
            TM = cv2.estimateAffinePartial2D(self.inputPts, outputPts)[0]
            self.img = cv2.warpAffine(self.img, TM, (workingSize,workingSize))

            #after transformation, resize to final output
            self.img = cv2.resize(self.img, (imgSize,imgSize), interpolation = cv2.INTER_AREA)

            #save final image
            cv2.imwrite(self.savePath, self.img)
        self.img = None
        self.displayImg = None
        
    def padImg(self, img):
        x, y, z = img.shape
        if x > y:
            pad = int((x - y)/2)
            img = cv2.copyMakeBorder(img, 0, 0, pad, ((x-y)-pad), cv2.BORDER_CONSTANT, None, value = 0)
        elif x < y:
            pad = int((y - x)/2)
            img = cv2.copyMakeBorder(img, pad, ((y-x)-pad), 0, 0, cv2.BORDER_CONSTANT, None, value = 0)
        return img

    def mouseEvent(self, event, x, y, flags, params):
        if event == cv2.EVENT_LBUTTONDOWN:
            self.inputPts[self.clickIndex][0] = x
            self.inputPts[self.clickIndex][1] = y    
            cv2.putText(self.displayImg, str(self.clickIndex), (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
            cv2.imshow('image', self.displayImg)
            self.clickIndex = 1 if self.clickIndex != 1 else 0 
        if event == cv2.EVENT_RBUTTONDOWN:
            self.inputPts = np.float32([[0,0],[0,0]])
            self.clickIndex = 0
            self.displayImg = copy.deepcopy(self.img)
            if self.model == True:
                cv2.putText(self.displayImg, 'Set Output Points', (20,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
            cv2.imshow('image', self.displayImg)

In [3]:
files = os.listdir(picsPath)
fileList = []

fileList.append(transformImage(modelImage))
    
for file in files:
    fileList.append(transformImage(file))
cv2.destroyAllWindows()